#### When the data might be stored in more than one vector store, we need to be able to route the query to the appropriate datasource to retrieve relevant docs.

### Logical Routing

We give the LLM knowledge of the various data sources at our disposal and then let the LLM reason which data source to apply based on the user’s query.
In order to achieve this, we make use of function-calling models like GPT-3.5 Turbo to help classify each query into one of the available routes.

In [33]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnableLambda, chain
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

In [10]:
# Data model
class RouteQuery(BaseModel):
    """Route a user's query to the most relevant datasource."""
    datasource: Literal["python_docs", "js_docs"] = Field(
        description="""Given users question, choose which data source is most relevant.""",
    )

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

system = """You are an expert at routing a user question to the relevant data source.
Based on the programming language the question is referring to, route it to the relevant data source.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

router = prompt | structured_llm

In [12]:
question = """Why doesn't the following code work: 
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question" : question})
result, result.datasource

(RouteQuery(datasource='python_docs'), 'python_docs')

Once we have the relevant datasource, we can route it to another function to execute additional logic.

In [15]:
def additiona_logic(result):
    if result.datasource.lower() == "python_docs":
        # logic here
        return "Routed to python docs"
    else:
        return "Routed to js docs"

# router_new = router | additiona_logic # works fine for simple functions (without async or strams)
router_new = router | RunnableLambda(additiona_logic) # explicitly make it runnable
router_new.invoke({"question" : question}) 

'Routed to python docs'

### Semantic Routing

In semantic routing, we take multiple prompts that represent various data sources, embed them alongside user's query and perform vector similarity search to retrieve the most similar prompt.

In [17]:
# prompts: prompt1 for physics questions, prompt2 for math questions

physics_template = """You are a very smart physics professor. You are great at
 answering questions about physics in a concise and easy-to-understand manner.
 When you don't know the answer to a question, you admit that you don't know.
Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering
 math questions. You are so good because you are able to break down hard
 problems into their component parts, answer the component parts, and then
 put them together to answer the broader question.
Here is a question:
{query}"""

In [21]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

In [23]:
len(prompt_embeddings[0])

1536

In [24]:
query = """What is a black hole"""

query_embedding = embeddings.embed_query(query)

In [25]:
len(query_embedding)

1536

In [28]:
similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
similarity

array([0.17683697, 0.12066461])

In [30]:
similar_prompt = prompt_templates[similarity.argmax()]
similar_prompt

"You are a very smart physics professor. You are great at\n answering questions about physics in a concise and easy-to-understand manner.\n When you don't know the answer to a question, you admit that you don't know.\nHere is a question:\n{query}"

In [31]:
PromptTemplate.from_template(similar_prompt)

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template="You are a very smart physics professor. You are great at\n answering questions about physics in a concise and easy-to-understand manner.\n When you don't know the answer to a question, you admit that you don't know.\nHere is a question:\n{query}")

In [ ]:
@chain
def prompt_router(query):
    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    similar_prompt = prompt_templates[similarity.argmax()]
    return PromptTemplate.from_template(similar_prompt)

In [ ]:
semantic_router = (
    prompt_router |
    ChatOpenAI(model="gpt-3.5-turbo") |
    StrOutputParser()
)

semantic_router.invoke(query)

In [42]:
query = "What can you tell me about Riemann Hypothesis?"

semantic_router.invoke(query)

'The Riemann Hypothesis is a famous unsolved problem in mathematics that was formulated by Bernhard Riemann in 1859. It deals with the distribution of prime numbers and states that all non-trivial zeros of the Riemann zeta function have a real part equal to 1/2. The Riemann zeta function is a complex function that is closely related to the distribution of prime numbers. \n\nThe Riemann Hypothesis is considered one of the most important unsolved problems in mathematics, and has important implications for number theory and the distribution of prime numbers. It is one of the seven Millennium Prize Problems identified by the Clay Mathematics Institute, with a reward of $1 million for its solution.\n\nWhile the Riemann Hypothesis has been extensively studied by mathematicians for over a century, no one has been able to prove it or disprove it. Many significant results in number theory have been obtained as a result of work on the Riemann Hypothesis, but it remains an open problem in mathema